In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import sys
from catboost import CatBoostRegressor

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#DATA
# df_users = pd.read_csv(r'/content/drive/My Drive/vkcup/users.tsv', delimiter='\t')
df_history = pd.read_csv(r'/content/drive/My Drive/vkcup/history.tsv', delimiter='\t')
df_test = pd.read_csv(r'/content/drive/My Drive/vkcup/validate.tsv', delimiter='\t')
# df_asc_feat = pd.read_csv(r'/content/drive/My Drive/vkcup/acs_feat.csv', index_col=0)
# df_hist_feats = pd.read_csv(r'/content/drive/My Drive/vkcup/df_hist_feats.csv', index_col=0)

In [ ]:
# def series_to_matrix(x, matrix):
#   n=0
#   for i in x:
#     for j in [int(k) for k in i.split(',')]:
#       # print(n,j-1)
#       matrix[n,j-1] = 1
#     n+=1

In [ ]:
# # BIG MATRIX CREATION
# publishers = np.zeros([DF0.shape[0], 21]).astype(int)
# series_to_matrix(DF0.publishers, publishers)
# print(publishers.shape)
# users = np.zeros([DF0.shape[0], 27768]).astype(int)
# series_to_matrix(DF0.user_ids, users)
# print(users.shape)

# DATA EXPLORATION

In [ ]:
# import seaborn as sns
# df_users = pd.read_csv(r'/content/drive/My Drive/vkcup/users.tsv', delimiter='\t')
# df_history = pd.read_csv(r'/content/drive/My Drive/vkcup/history.tsv', delimiter='\t')
# df_test = pd.read_csv(r'/content/drive/My Drive/vkcup/validate.tsv', delimiter='\t')

In [ ]:
# days = df_test.hour_start//24
# sns.lineplot(days.value_counts().index, days.value_counts())

In [ ]:
# days = df_test.hour_end//24
# sns.lineplot(days.value_counts().index, days.value_counts())

In [ ]:
# # PUBLISHERS HISTORY U VALIDATE
# df_publisher_dummies = pd.get_dummies(df_history['publisher'])
# df_history1 = df_history.merge(df_publisher_dummies, left_index=True, right_index=True)
# hist_feats1 = df_history1[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,'user_id']].groupby('user_id').sum().astype(int)

In [ ]:
# def func_hist_publ(string,col):
#   q = np.array([int(i) for i in string.split(',')])
#   return hist_feats1.loc[q,col].sum()

# df_func_hist_publ = pd.DataFrame(np.zeros([1008, 21]).astype(int), columns=np.array(np.arange(1,22)))

# for p in range(1,22):
#   df_func_hist_publ.iloc[:, p-1] = df_test.user_ids.apply(func_hist_publ, col=p)

# df_hist_feats1 = pd.DataFrame(df_func_hist_publ.astype(int))
# df_hist_feats1 = df_hist_feats1.merge(df_test['publishers'].apply(lambda x: [int(i) for i in x.split(',')]), left_index=True, right_index=True)

In [ ]:
# def func_publ_ratio(series):
#   pr = []
#   for i in series['publishers']:
#     pr.append(series[i])
#   return sum(pr)/series[:-1].sum()

# publ_ratio = df_hist_feats1.apply(func_publ_ratio, axis=1)
# p_one = df_hist_feats1['publishers'].apply(lambda x: sum(1 for i in x if i==1))
# df_publishers = pd.DataFrame({'publ_ratio':publ_ratio, 'p_one': p_one})
# df_publishers


# HISTORY FEATURE ENGINEERING

In [ ]:
#DELTA HOUR
delta_hour = df_test.hour_end - df_test.hour_start
df_delta_hour = pd.DataFrame(delta_hour, columns = ['delta_hour']) 
# #audience_size/delta
# audsize_delta = (df_delta_hour.values.reshape(1,-1))/(DF0.audience_size.values.reshape(1,-1))
# df_audsize_delta = pd.DataFrame(audsize_delta.reshape(-1,1),  columns = ['audsize_delta']) 
# #cpm*delta
# cpm_delta = df_delta_hour.values.reshape(1,-1)*DF0.cpm.values.reshape(1,-1)
# df_cpm_delta = pd.DataFrame(cpm_delta.reshape(-1,1), columns = ['cpm_delta']) 

In [ ]:
hist_feats = df_history[['cpm','hour','user_id']].groupby('user_id').agg({'cpm':'mean', 'hour':'count'}).\
  rename(columns={'cpm':'cpm_mean', 'hour':'show_counts'})

In [ ]:
# HISTORY CPM & COUNT NEW
def func_hist_cpm(string):
  q = np.array([int(i) for i in string.split(',')])
  return hist_feats.loc[q,'cpm_mean'].sum()/len(q)
def func_hist_count(string):
  q = np.array([int(i) for i in string.split(',')])
  return hist_feats.loc[q,'show_counts'].sum()/len(q)

df_func_hist_cpm = df_test.user_ids.apply(func_hist_cpm)
df_func_hist_count = df_test.user_ids.apply(func_hist_count)
df_hist_feats = pd.DataFrame({'0':df_func_hist_cpm, '1': df_func_hist_count})

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [ ]:
# PUBLISHERS FEATURES
df_publisher_dummies = pd.get_dummies(df_history['publisher'])
df_history1 = df_history.merge(df_publisher_dummies, left_index=True, right_index=True)
hist_feats1 = df_history1[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,'user_id']].groupby('user_id').sum().astype(int)

In [ ]:
def func_hist_publ(string,col):
  q = np.array([int(i) for i in string.split(',')])
  return hist_feats1.loc[q,col].sum()

df_func_hist_publ = pd.DataFrame(np.zeros([df_test.shape[0], 21]).astype(int), columns=np.array(np.arange(1,22)))

for p in range(1,22):
  df_func_hist_publ.iloc[:, p-1] = df_test.user_ids.apply(func_hist_publ, col=p)

df_hist_feats1 = pd.DataFrame(df_func_hist_publ.astype(int))
df_hist_feats1 = df_hist_feats1.merge(df_test['publishers'].apply(lambda x: [int(i) for i in x.split(',')]), left_index=True, right_index=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [ ]:
def func_publ_ratio(series):
  pr = []
  for i in series['publishers']:
    pr.append(series[i])
  return sum(pr)/series[:-1].sum()

publ_ratio = df_hist_feats1.apply(func_publ_ratio, axis=1)
p_one = df_hist_feats1['publishers'].apply(lambda x: sum(1 for i in x if i==1))
df_publishers = pd.DataFrame({'publ_ratio':publ_ratio, 'p_one': p_one})

In [ ]:
# TIME FEATURES
df_history['days'] = df_history.hour//24
workdays = set(range(0,200)) - (set(range(5,200,7)) | set(range(6,200,7)))
holidays = set(range(5,200,7)) | set(range(6,200,7))
df_history['workdays'] = df_history['days'].apply(lambda x: 1 if x in workdays else 0)
df_history['holidays'] = df_history['days'].apply(lambda x: 1 if x in holidays else 0)

In [ ]:
df_test['days_start'] = df_test.hour_start//24
df_test['days_end'] = df_test.hour_end//24
def func_days_test(series):
  return list(range(series.loc['days_start'], series.loc['days_end']+1))
df_test['days'] = df_test.apply(func_days_test, axis=1)
def func_ratio_workdays(days):
  wd = sum(1 for i in days if i in workdays)
  return wd/len(days)
df_test['ratio_workdays'] = df_test['days'].apply(func_ratio_workdays)

df_hist2 = df_history[['workdays', 'holidays','user_id']].groupby('user_id').sum().astype(int)
df_hist2['all_days'] = df_hist2.workdays+df_hist2.holidays
df_hist2['rratio'] = df_hist2.workdays/df_hist2.all_days
df_hist2['aratio'] = df_hist2['rratio']*df_hist2.workdays

In [ ]:
def func_hist2_count(string):
  q = np.array([int(i) for i in string.split(',')])
  return df_hist2.loc[q,'aratio'].sum()/len(q)

df_func_hist2_count = df_test.user_ids.apply(func_hist2_count)
df_func_hist2_count = df_test.user_ids.apply(func_hist_count)
df_hist2_feats = pd.DataFrame({'history_workdays_ratio':df_func_hist2_count})

df_test.drop(['days_start','days', 'days_end'], inplace=True, axis=1)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


# SUMMARY

In [ ]:
df_test1 = df_test.drop(['publishers', 'user_ids'], axis=1)
df_test1 = df_test1.drop(['hour_start', 'hour_end'], axis=1)
df_test1 = df_test1.merge(df_delta_hour, left_index=True, right_index=True)
df_test1 = df_test1.merge(df_hist_feats, left_index=True, right_index=True)
df_test1 = df_test1.merge(df_publishers, left_index=True, right_index=True)

df_test1 = df_test1.merge(df_hist2_feats, left_index=True, right_index=True)
df_test1['mm'] = df_test1.ratio_workdays*df_test1.history_workdays_ratio
# DF1 = DF1.merge(df_audsize_delta, left_index=True, right_index=True)
# DF1 = DF1.merge(df_cpm_delta, left_index=True, right_index=True)

# MERTIC

In [ ]:
# # METRIC
# def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
#     return np.abs(np.log((responses_column + epsilon)/(answers_column + epsilon) )).mean()


# def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
#     log_accuracy_ratio_mean = np.array(
#         [   get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
#             get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
#             get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon)]).mean()

#     percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)
#     return percentage_error.round(decimals=2)
# answers = pd.read_csv(r'/content/drive/My Drive/vkcup/validate_answers.tsv', delimiter='\t')
# print(get_smoothed_mean_log_accuracy_ratio(answers, prediction))

# FIT MODELS BLOCK

In [ ]:
# # CROSS-VALIDATION INDEXES
# n = set(range(1008))
# fold1_v = list(np.random.choice(list(n), 252, replace=False))
# n -= set(fold1_v)
# fold1_t = list(set(range(1008))-set(fold1_v))

# fold2_v = list(np.random.choice(list(n), 252, replace=False))
# n -= set(fold2_v)
# fold2_t = list(set(range(1008))-set(fold2_v))

# fold3_v = list(np.random.choice(list(n), 252, replace=False))
# n -= set(fold3_v)
# fold3_t = list(set(range(1008))-set(fold3_v))

# fold4_v = list(n)
# fold4_t = list(set(range(1008))-set(fold4_v))
# folds = ((fold1_t, fold1_v), (fold2_t, fold2_v), (fold3_t, fold3_v), (fold4_t, fold4_v),)

In [ ]:
# # CROSS-VALIDATION SCORE
# score = []
# for i in folds:
#   ctr1 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAE')
#   ctr2 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAE')
#   ctr3 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAE')

#   ctr1.fit(df_test1.iloc[i[0],:], answers.at_least_one.iloc[i[0]], eval_set=(df_test1.iloc[i[1],:], answers.at_least_one.iloc[i[1]]), verbose=False)
#   ctr2.fit(df_test1.iloc[i[0],:], answers.at_least_two.iloc[i[0]], eval_set=(df_test1.iloc[i[1],:], answers.at_least_two.iloc[i[1]]), verbose=False)
#   ctr3.fit(df_test1.iloc[i[0],:], answers.at_least_three.iloc[i[0]], eval_set=(df_test1.iloc[i[1],:], answers.at_least_three.iloc[i[1]]), verbose=False)

#   pred1 = ctr1.predict(df_test1).clip(0.0000001,1)
#   pred2 = ctr2.predict(df_test1).clip(0.0000001,1)
#   pred3 = ctr3.predict(df_test1).clip(0.0000001,1)
#   predictions = pd.DataFrame({'at_least_one':pred1,'at_least_two':pred2, 'at_least_three':pred3})
#   print(ctr1.best_score_, ctr1.tree_count_)
#   print(ctr2.best_score_, ctr2.tree_count_)
#   print(ctr3.best_score_, ctr3.tree_count_)
#   print(get_smoothed_mean_log_accuracy_ratio(answers, predictions))
#   score.append(get_smoothed_mean_log_accuracy_ratio(answers, predictions))
# print(score, sum(score)/4)

In [ ]:
# # FULL_DF FIT/PREDICT
# ctr1 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAE')
# ctr2 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAE')
# ctr3 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAE')
# ctr1.fit(df_test1, answers.at_least_one, verbose=False)
# ctr2.fit(df_test1, answers.at_least_two, verbose=False)
# ctr3.fit(df_test1, answers.at_least_three, verbose=False)
# pred1 = ctr1.predict(df_test1).clip(0.0000001,1)
# pred2 = ctr2.predict(df_test1).clip(0.0000001,1)
# pred3 = ctr3.predict(df_test1).clip(0.0000001,1)
# prediction = pd.DataFrame({'at_least_one':pred1,'at_least_two':pred2, 'at_least_three':pred3})
# print(get_smoothed_mean_log_accuracy_ratio(answers, prediction))

7.82


In [ ]:
# # SAVE MODELS
# ctr1.save_model(r'/content/drive/My Drive/vkcup/ctr1')
# ctr2.save_model(r'/content/drive/My Drive/vkcup/ctr2')
# ctr3.save_model(r'/content/drive/My Drive/vkcup/ctr3')

# PREDICTION

In [ ]:
# LOAD MODELS
ctr1 = CatBoostRegressor(early_stopping_rounds=10)
ctr2 = CatBoostRegressor(early_stopping_rounds=10)
ctr3 = CatBoostRegressor(early_stopping_rounds=10)
ctr1.load_model(r'/content/drive/My Drive/vkcup/ctr1')
ctr2.load_model(r'/content/drive/My Drive/vkcup/ctr2')
ctr3.load_model(r'/content/drive/My Drive/vkcup/ctr3')

In [ ]:
# PREDICTION
pred10 = ctr1.predict(df_test1).clip(0.0000001,1)
pred20 = ctr2.predict(df_test1).clip(0.0000001,1)
pred30 = ctr3.predict(df_test1).clip(0.0000001,1)
prediction = pd.DataFrame({'at_least_one':pred10,'at_least_two':pred20, 'at_least_three':pred30})

In [ ]:
# SAVE RESULT
prediction.to_csv(r'/content/drive/My Drive/vkcup/results.tsv', sep='\t', index=False, header=True)